In [ ]:
import os
import re
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

import math
import matplotlib.colors as cx
import matplotlib.cm as cm

from glob import glob
from scipy import signal, stats
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

pd.options.mode.chained_assignment = None
%matplotlib inline

In [ ]:
path = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\snow_flies'
outpath = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\summaries'
outfile = os.path.join(outpath, 'snowfly_data_final.parquet')
savepath = r'G:\My Drive\Tuthill Lab Shared\Katie\presentations\lab_meeting\data\2021_06_15\figures'

In [ ]:
data = pd.DataFrame()
(root, dirs, files) = next(os.walk(path))

for session in dirs: 
    
    session_path = os.path.join(path, session)
    (r, flies, f) = next(os.walk(session_path))
    
    for fly in flies:
        
        fly_path = os.path.join(session_path, fly)
        (r, trials, f) = next(os.walk(fly_path))
        
        for trial in trials: 
            
            corr_path = os.path.join(fly_path, trial, 'temp_data_corrections_visible.csv')
            trial_path = os.path.join(fly_path, trial, 'temp_data_final.csv')
            video_path = glob(os.path.join(fly_path, trial, '*.csq'))
            if len(video_path) == 0:
                continue
                
            # print(video_path)
            video_name = os.path.basename(video_path[0])
            if not os.path.exists(trial_path):
                continue
                
            print(trial_path)
            
            corr_data = pd.read_csv(corr_path)
            trial_num = re.findall(r'\d+', trial)[0]
            trial_data = pd.read_csv(trial_path)
            trial_data['date'] = session
            trial_data['fly'] = fly
            trial_data['visible'] = corr_data.visible
            trial_data['trial'] = trial_num
            trial_data['flyid'] = fly + '_' + trial_num
            trial_data['fnum'] = np.arange(0, len(trial_data), 1)
            trial_data['filename'] = video_name
            trial_data['fullfile'] = os.path.join(session, fly, trial, video_name)
            data = data.append(trial_data)
            
# save data to parquet file
data.loc[~data.visible, ['x', 'y', 'x_filt', 'y_filt', 'max_temp', 'intensity', 'movement']] = np.nan
data.to_parquet(outfile + '.gzip', compression = 'gzip')

In [ ]:
data = pd.read_parquet(outfile + '.gzip', engine='fastparquet')

In [ ]:
data_old = pd.read_parquet(r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\summaries\snowfly_data.parquet' + '.gzip', engine='fastparquet')

In [ ]:
def get_video_length(nframes, thresh = None, fps = 30):

    frames = np.arange(0, nframes, 1)
    time_s = frames / fps
    time_m = frames / (fps*60)
    
    if thresh is not None:
        time_s = time_s[:thresh]
        time_m = time_m[:thresh]
         
    return time_s, time_m

def nan_helper(y):
    return np.isnan(y), lambda z: z.nonzero()[0]

In [ ]:
# interpolate x and y positions (replacing tracking errors by interpolating)
flyids = np.unique(data.flyid)
thresh = 35

for j in range(len(flyids)):
    mask = data.flyid == flyids[j]
    fly_df = data.loc[mask]
    x_filtered = signal.medfilt(fly_df.x_filt, kernel_size = 151)
    y_filtered = signal.medfilt(fly_df.y_filt, kernel_size = 151)
    x_err = np.abs(x_filtered - fly_df.x_filt)
    y_err = np.abs(y_filtered - fly_df.y_filt)
    errors = (x_err > thresh) | (y_err > thresh)
    fly_df.loc[errors, 'x_filt'] = np.nan
    fly_df.loc[errors, 'y_filt'] = np.nan
    
    nans, ix = nan_helper(np.array(fly_df.x_filt))
    fly_df.loc[nans, 'x_filt'] = np.interp(ix(nans), ix(~nans), fly_df.loc[~nans, 'x_filt'])
    fly_df.loc[nans, 'y_filt'] = np.interp(ix(nans), ix(~nans), fly_df.loc[~nans, 'y_filt'])
    
    data.loc[mask, 'x_interp'] = fly_df.x_filt
    data.loc[mask, 'y_interp'] = fly_df.y_filt
    data.loc[mask, 'tracking_error'] = errors
    
data.loc[~data.visible, 'x_interp'] = np.nan
data.loc[~data.visible, 'y_interp'] = np.nan

In [ ]:
# median filter for x and y positions (replace tracking errors with median filter)
flyids = np.unique(data.flyid)
thresh = 35

for j in range(len(flyids)):
    mask = data.flyid == flyids[j]
    fly_df = data.loc[mask]
    x_filtered = signal.medfilt(fly_df.x_filt, kernel_size = 151)
    y_filtered = signal.medfilt(fly_df.y_filt, kernel_size = 151)
    x_err = np.abs(x_filtered - fly_df.x_filt)
    y_err = np.abs(y_filtered - fly_df.y_filt)
    errors = (x_err > thresh) | (y_err > thresh)
    fly_df.loc[errors, 'x_filt'] = x_filtered[errors]
    fly_df.loc[errors, 'y_filt'] = y_filtered[errors]
    data.loc[mask, 'x_filtered'] = fly_df.x_filt
    data.loc[mask, 'y_filtered'] = fly_df.y_filt
    # data.loc[mask, 'tracking_error'] = errors

In [ ]:
# fully filtered positions 
flyids = np.unique(data.flyid)

for j in range(len(flyids)):
    
    mask = data.flyid == flyids[j]
    fly_df = data.loc[mask]
    x_filtered_full = signal.medfilt(fly_df.x_interp, kernel_size = 15)  
    y_filtered_full = signal.medfilt(fly_df.y_interp, kernel_size = 15)
    data.loc[mask, 'x_filtered_full'] = x_filtered_full
    data.loc[mask, 'y_filtered_full'] = y_filtered_full


In [ ]:
sns.set()
sns.set_style('ticks')
savepath = r'G:\My Drive\Tuthill Lab Shared\Katie\presentations\lab_meeting\data\2021_06_15\figures'

flyids = np.intersect1d(np.unique(data.flyid), np.unique(data_old.flyid))
flyids = ['SF0039_1']
colors = sns.color_palette("husl", 8)
dpi = 600

for j in range(len(flyids)):
    
    mask = data.flyid == flyids[j]
    fly_df = data.loc[mask]
    mask_old = data_old.flyid == flyids[j]
    fly_df_old = data_old.loc[mask_old]
    x_filtered_full = signal.medfilt(fly_df.x_interp, kernel_size = 15)
    y_filtered_full = signal.medfilt(fly_df.y_interp, kernel_size = 15)  
    t_s, t_m = get_video_length(len(fly_df))
    
    fig = plt.figure(figsize = (6, 4), dpi = dpi)
    plt.title('raw position data', fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('x position', fontsize = 14)
    plt.plot(t_m, fly_df_old.x, color = '#b66a50', linewidth = 0.5)
    plt.ylim([100, 600])
    sns.despine()
    plt.savefig(os.path.join(savepath, 'tracking_1a.png'))
    plt.show()
    
    fig = plt.figure(figsize = (4, 4), dpi = dpi)
    ax = plt.gca()
    plt.title('raw 2d trajectory', fontsize = 14)
    xs = fly_df_old.x #[:ix]
    ys = fly_df_old.y # [:ix]
    ax.plot(ys, xs, color = '#b66a50', linewidth = 0.5)  
    ax.axis('equal')
    sns.despine()
    ax.invert_yaxis()
    plt.axis('off')
    plt.savefig(os.path.join(savepath, 'tracking_1b.png'))
    plt.show()
    
    fig = plt.figure(figsize = (6, 4), dpi = dpi)
    plt.title('position data after manual correction', fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('x position', fontsize = 14)
    plt.plot(t_m, fly_df.x, color = '#bf9005', linewidth = 0.5)
    plt.ylim([100, 600])
    sns.despine()
    plt.savefig(os.path.join(savepath, 'tracking_2a.png'))
    plt.show()
    
    fig = plt.figure(figsize = (4, 4), dpi = dpi)
    ax = plt.gca()
    plt.title('2d trajectory after manual correction', fontsize = 14)
    xs = fly_df.x #[:ix]
    ys = fly_df.y # [:ix]
    ax.plot(ys, xs, color = '#bf9005', linewidth = 0.5)  
    ax.axis('equal')
    sns.despine()
    ax.invert_yaxis()
    plt.axis('off')
    plt.savefig(os.path.join(savepath, 'tracking_2b.png'))
    plt.show()
    
    fig = plt.figure(figsize = (6, 4), dpi = dpi)
    plt.title('position data after filtering and interpolating', fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('x position', fontsize = 14)
    plt.plot(t_m, x_filtered_full, color = colors[3], linewidth = 0.5)
    plt.ylim([100, 600])
    sns.despine()
    plt.savefig(os.path.join(savepath, 'tracking_3a.png'))
    plt.show()
    
    fig = plt.figure(figsize = (4, 4), dpi = dpi)
    ax = plt.gca()
    plt.title('2d trajectory after filtering and interpolating', fontsize = 14)
    xs = x_filtered_full #[:ix]
    ys = y_filtered_full # [:ix]
    ax.plot(ys, xs, color = colors[3], linewidth = 0.5)  
    ax.axis('equal')
    sns.despine()
    ax.invert_yaxis()
    plt.axis('off')
    plt.savefig(os.path.join(savepath, 'tracking_3b.png'))
    plt.show()
    
    #data.loc[mask, 'x_filtered_full'] = x_filtered_full
    #data.loc[mask, 'y_filtered_full'] = y_filtered_full


In [ ]:
# compute velocity, acceleration 
# normalize according to the diameter of the metal ring
fps = 30
dt = 1 / fps
s = 1.0 / dt
s2 = 1.0 / (dt * dt)
flyids = np.unique(data.flyid)
diameter_true = 70 # mm

for j in range(len(flyids)):
    
    fly_mask = data.flyid == flyids[j]
    fly_df = data.loc[fly_mask]
    
    trial_path = os.path.dirname(fly_df.fullfile.iloc[0])
    mask_path = os.path.join(path, trial_path, 'mask - Ellipse 1.bmp')
    mask = cv2.imread(mask_path, cv2.THRESH_BINARY)
    mask = mask / np.max(mask)
    sum_x = np.sum(mask, axis = 0)
    sum_y = np.sum(mask, axis = 1)
    diameter_px = int(np.nanmean([np.max(sum_x), np.max(sum_y)]))
    norm = diameter_true / diameter_px
    
    xs = fly_df.x_filtered_full
    ys = fly_df.y_filtered_full
    positions = np.vstack((xs, ys))
    
    data.loc[fly_mask, 'velocity'] = np.nan
    data.loc[fly_mask, 'acceleration'] = np.nan
    finite = fly_mask
    pos_mask = np.isfinite(xs) & np.isfinite(ys)
    finite[fly_mask] = pos_mask
    
    velocity = signal.savgol_filter(positions[:, pos_mask], 901, 3, deriv=1, axis = 1) * s * norm
    speed = np.linalg.norm(velocity, axis = 0)   
    
#     fig = plt.figure()
#     plt.title(flyids[j])
#     plt.plot(speed)
#     plt.show()
    
    acceleration = signal.savgol_filter(positions[:, pos_mask], 901, 3, deriv=2, axis = 1) * s2 * norm
    acc = np.linalg.norm(acceleration, axis = 0) 
    data.loc[finite, 'velocity'] = np.array(speed) 
    data.loc[finite, 'acceleration'] = np.array(acc)

In [ ]:
# median filter for distance, velocities, and acceleration 
flyids = np.unique(data.flyid)
v_thresh = 0.5

for j in range(len(flyids)):
    
    mask = data.flyid == flyids[j]
    fly_df = data.loc[mask]
    
    v_filt = signal.medfilt(abs(fly_df.velocity), kernel_size = 201)
    err = np.abs(v_filt - fly_df.velocity) 
    err = np.abs(fly_df.velocity)

    errors = (err > v_thresh)
    fly_df.loc[errors, 'velocity'] = v_filt[errors]
    data.loc[mask, 'velocity_filtered'] = fly_df.velocity
    
    a_filt = signal.medfilt(abs(fly_df.acceleration), kernel_size = 201)
    fly_df.loc[errors, 'acceleration'] = a_filt[errors]
    data.loc[mask, 'acceleration_filtered'] = fly_df.acceleration
    
    fig = plt.figure(figsize = (8,4))
    plt.title(flyids[j])
    plt.plot(np.abs(data.loc[mask, 'velocity']), 'r', linewidth = 0.5)
    # plt.plot(v_filt, 'k')
    plt.plot(np.abs(data.loc[mask, 'velocity_filtered']), 'k', linewidth = 0.5)
    sns.despine()
    plt.show()

In [ ]:
# for each fly, determine the 75th percentile speed
flyids = np.unique(data.flyid)
data['exclude'] = False
pct = 75
thresh = 0.00001 # mm/s

for j in range(len(flyids)):
    
    mask = data.flyid == flyids[j]
    fly_df = data.loc[mask]
    
    velocity = fly_df.velocity[np.isfinite(fly_df.velocity)]
    p = np.percentile(velocity, pct)
    
    if p <= thresh:
        print(flyids[j])
        data.loc[mask, 'exclude'] = True

In [ ]:
# distribution of snow fly velocity
sns.set()
sns.set_style('ticks')

fig = plt.figure(figsize = (9, 5))
plt.title('distribution of snow fly velocities', fontsize = 14)
plt.ylabel('probability density', fontsize = 14)
plt.xlabel('speed (mm/s)', fontsize = 14)

velocities = data.velocity[np.isfinite(data.velocity)]
kernel = stats.gaussian_kde(velocities)
points = np.linspace(np.percentile(velocities, 0), np.percentile(velocities, 99), 100)
height = kernel.pdf(points)
plt.plot(points, height, label = 'cold plate temperature', color = colors[0])

sns.despine()
plt.show()

In [ ]:
# histogram of velocity (all flies)
sns.set()
sns.set_style('ticks')

figure = plt.figure(figsize = (9, 5))
plt.title('distribution of snow fly velocities', fontsize = 14)
plt.ylabel('probability density', fontsize = 14)
plt.xlabel('velocity (mm/s)', fontsize = 14)

velocities = np.abs(data.velocity[np.isfinite(data.velocity)])
v = velocities[velocities < np.percentile(velocities, 95)]
plt.hist(v, bins = 15, color = 'k')

sns.despine()
plt.show()

In [ ]:
# plot velocitity for presentation
sns.set()
sns.set_style('ticks')
flies = ['SF0039_1']
max_frames = 54000
dpi = 600
c = 'xkcd:greeny blue'
c = 'xkcd:violet'

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (6, 4), dpi = dpi)
    ax = plt.gca()
    plt.title('snow fly movement', fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('absolute velocity (mm/s)', fontsize = 14)
    ax.plot(t_m[200:], fly_data.velocity[200:len(t_m)], 'xkcd:black', linewidth = 1, zorder = 1)
    
    ax2 = ax.twinx()
    ax2.set_ylabel('cold plate temperature ($^{\circ}$C)', fontsize = 14, labelpad = 10, color = c)
    ax2.plot(t_m[200:], fly_data.cold_plate_temp_filtered[200:len(t_m)], c, zorder = 2, alpha = 0.6)
    ax2.spines['right'].set_color(c)
    ax2.tick_params(axis='y', colors=c)
    
    sns.despine(right = False)
    plt.tight_layout()
    plt.savefig(os.path.join(savepath, 'velocity_1.png'), dpi=1000)
    plt.show()

In [ ]:
# plot velocities for each fly
sns.set()
sns.set_style('ticks')
flies = only_cc #np.unique(data.flyid)
max_frames = 54000
window_size = 151

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (8, 4))
    plt.title(fly, fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('absolute velocity (mm/s)', fontsize = 14)
    running_mean = np.convolve(fly_data.velocity[:len(t_m)], np.ones(window_size)/window_size, mode='valid')
    plt.plot(t_m, fly_data.velocity[:len(t_m)], 'xkcd:black', linewidth = 0.5)
    # plt.plot(t_m[int(window_size/2):-int(window_size/2)], running_mean, 'xkcd:blue', linewidth = 0.5)
    sns.despine()
    plt.show()

In [ ]:
# plot acceleration for each fly
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
max_frames = 54000
window_size = 151

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (8, 4))
    plt.title(fly, fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('absolute acceleration (mm/s$^{2}$)', fontsize = 14)
    abs_acceleration_filtered = np.abs(fly_data.acceleration_filtered[:len(t_m)])
    running_mean = np.convolve(abs_acceleration_filtered, np.ones(window_size)/window_size, mode='valid')
    plt.plot(t_m, abs_acceleration_filtered, 'xkcd:black', linewidth = 0.5)
    plt.plot(t_m[int(window_size/2):-int(window_size/2)], running_mean, 'xkcd:blue', linewidth = 0.5)
    sns.despine()
    plt.show()

In [ ]:
# interpolate temperatures where there are tracking errors 
flyids = np.unique(data.flyid)
    
for j in range(len(flyids)):
    
    mask = data.flyid == flyids[j]
    fly_df = data.loc[mask]
    errors = fly_df.tracking_error & fly_df.visible
    
    fly_df.loc[errors, 'avg_temp'] = np.nan
    fly_df.loc[errors, 'max_temp'] = np.nan
    fly_df.loc[errors, 'cold_plate_temp'] = np.nan
    
    nans, ix = nan_helper(np.array(fly_df.avg_temp))
    fly_df.loc[nans, 'avg_temp'] = np.interp(ix(nans), ix(~nans), fly_df.loc[~nans, 'avg_temp'])
    fly_df.loc[nans, 'max_temp'] = np.interp(ix(nans), ix(~nans), fly_df.loc[~nans, 'max_temp'])
    fly_df.loc[nans, 'cold_plate_temp'] = np.interp(ix(nans), ix(~nans), fly_df.loc[~nans, 'cold_plate_temp'])
    
    data.loc[mask, 'avg_temp_interp'] = fly_df['avg_temp']
    data.loc[mask, 'max_temp_interp'] = fly_df['max_temp']
    data.loc[mask, 'cold_plate_temp_interp'] = fly_df['cold_plate_temp']

In [ ]:
# median filter for temperatures
flyids = np.unique(data.flyid)
# flyids = ['SF0039_1']

for j in range(len(flyids)):
    mask = data.flyid == flyids[j]
    fly_df = data.loc[mask]
    cp_temps_filtered = signal.medfilt(fly_df.cold_plate_temp_interp, kernel_size = 151)
    avg_temps_filtered = signal.medfilt(fly_df.avg_temp_interp, kernel_size = 151)
    max_temps_filtered = signal.medfilt(fly_df.max_temp_interp, kernel_size = 151)
    
#     fig = plt.figure(figsize = (6, 4))
#     plt.plot(cp_temps_filtered, linewidth = 0.5)
#     plt.plot(avg_temps_filtered, linewidth = 0.5)
#     plt.plot(max_temps_filtered, linewidth = 0.5)
#     plt.show()

    data.loc[mask, 'cold_plate_temp_filtered'] = cp_temps_filtered
    data.loc[mask, 'avg_temp_filtered'] = avg_temps_filtered
    data.loc[mask, 'max_temp_filtered'] = max_temps_filtered

In [ ]:
# detect temperature increases
flyids = np.unique(data.flyid)
# flyids = ['SF0039_1']

for j in range(len(flyids)):
    mask = data.flyid == flyids[j]
    fly_df = data.loc[mask]
    cp_temps_filtered = signal.medfilt(fly_df.cold_plate_temp_interp, kernel_size = 15)
    avg_temps_filtered = signal.medfilt(fly_df.avg_temp_interp, kernel_size = 15)
    max_temps_filtered = signal.medfilt(fly_df.max_temp_interp, kernel_size = 15)
    
    fig = plt.figure(figsize = (6, 4))
    plt.plot(cp_temps_filtered, linewidth = 0.5)
    plt.plot(avg_temps_filtered, linewidth = 0.5)
    plt.plot(max_temps_filtered, linewidth = 0.5)
    plt.show()

#     data.loc[mask, 'cold_plate_temp_filtered'] = cp_temps_filtered
#     data.loc[mask, 'avg_temp_filtered'] = avg_temps_filtered
#     data.loc[mask, 'max_temp_filtered'] = max_temps_filtered

In [ ]:
# temperature example for presentation 
sns.set()
sns.set_style('ticks')
flies = ['SF0039_1']
max_frames = 54000
dpi = 600

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    fly_data_old = data_old[data_old.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (6, 4), dpi = dpi)
    ax = plt.gca()
    plt.title('raw temperature data', fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('temperature ($^{\circ}$C)', fontsize = 14)
    plt.plot(t_m, fly_data_old.max_temp[:len(t_m)], 'xkcd:light blue', linewidth = 0.5, label = 'detected temp')
    plt.plot(t_m, fly_data_old.avg_temp[:len(t_m)], 'xkcd:slate blue', linewidth = 0.5, label = 'avg temp')
    plt.plot(t_m, fly_data_old.cold_plate_temp[:len(t_m)], 'xkcd:black', linewidth = 0.5, label = 'cold plate temp') 
    sns.despine()
    plt.savefig(os.path.join(savepath, 'temp_1a.png'))
    plt.show()
    
    fig = plt.figure(figsize = (6, 4), dpi = dpi)
    ax = plt.gca()
    plt.title('temperature data after filtering and interpolating', fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('temperature ($^{\circ}$C)', fontsize = 14) 
    plt.plot(t_m, fly_data.max_temp_filtered[:len(t_m)], 'xkcd:light blue', linewidth = 1, label = 'detected snow fly temp')
    plt.plot(t_m, fly_data.avg_temp_filtered[:len(t_m)], 'xkcd:slate blue', linewidth = 1, label = 'average snow fly temp')
    plt.plot(t_m, fly_data.cold_plate_temp_filtered[:len(t_m)], 'xkcd:black', linewidth = 1, label = 'cold plate temp')
    hs, ls = ax.get_legend_handles_labels()
    sns.despine()
    plt.savefig(os.path.join(savepath, 'temp_1b.png'))
    plt.show()
    
    fig = plt.figufigsize = (3,3), dpi = dpi)
    plt.legend(handles = hs, labels = ls, fontsize = 12)
    plt.axis('off')
    plt.savefig(os.path.join(savepath, 'temp_legend.png'))
    plt.show()

In [ ]:
# temperature example for presentation 
sns.set()
sns.set_style('ticks')
flies = ['SF0051_1']
max_frames = 54000
dpi = 600

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    fly_data_old = data_old[data_old.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (6, 4), dpi = dpi)
    ax = plt.gca()
    # plt.title('partial freezing example', fontsize = 14)
    # plt.title('cold room example trial', fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('temperature ($^{\circ}$C)', fontsize = 14) 
    plt.plot(t_m, fly_data.max_temp_filtered[:len(t_m)], 'xkcd:light blue', linewidth = 1, label = 'detected snow fly temp')
    plt.plot(t_m, fly_data.avg_temp_filtered[:len(t_m)], 'xkcd:slate blue', linewidth = 1, label = 'average snow fly temp')
    plt.plot(t_m, fly_data.cold_plate_temp_filtered[:len(t_m)], 'xkcd:black', linewidth = 1, label = 'cold plate temp')
    hs, ls = ax.get_legend_handles_labels()
    sns.despine()
    plt.tight_layout()
    # plt.savefig(os.path.join(savepath, 'temp_sf0100.png'))
    # plt.savefig(os.path.join(savepath, 'temp_sf1104_partial_freezing.png'))
    plt.show()


In [ ]:
# calculate average velocities for bins of temperature (all flies)
flyids = np.unique(data.flyid)
delta = 0.5
bins = np.arange(-18, 18, delta)
v_df = pd.DataFrame()
v_thresh = 0

for i, fly in enumerate(flyids):
    
    fly_data = data[data.flyid == fly]
    
    ix = np.argmin(fly_data.cold_plate_temp_filtered)
    abs_velocity = np.abs(fly_data.velocity)
    velocities_c = np.array(abs_velocity[:ix])
    temperatures_c = np.array(fly_data.cold_plate_temp_filtered[:ix])
    velocities_w = np.array(abs_velocity[ix:])
    temperatures_w = np.array(fly_data.cold_plate_temp_filtered[ix:])
    
    mask_c = velocities_c >= v_thresh
    mask_w = velocities_w >= v_thresh
    velocities_c = velocities_c[mask_c]
    velocities_w = velocities_w[mask_w]
    temperatures_c = temperatures_c[mask_c]
    temperatures_w = temperatures_w[mask_w]

    v_c = np.zeros([len(bins)-1])
    v_w = np.zeros([len(bins)-1])
    
    for j in range(len(bins)-1): 
        temp_mask_c = (temperatures_c >= bins[j]) & (temperatures_c < bins[j+1])
        v_c[j] = np.nanmean(velocities_c[temp_mask_c])
        temp_mask_w = (temperatures_w >= bins[j]) & (temperatures_w < bins[j+1])
        v_w[j] = np.nanmean(velocities_w[temp_mask_w])
        
    row = {'flyid': fly, 'velocity_cooling': v_c, 'velocity_warming': v_w}
    v_df = v_df.append(row, ignore_index = True)

In [ ]:
from matplotlib.ticker import MaxNLocator

sns.set()
sns.set_style('ticks')
dpi = 600

t = np.arange(0, 1201, 150)
t_m = t / 60
temps = np.array([20, 0, -2, -4, -6, -8, -10, -12, -14])

fig = plt.figure(figsize = (6, 3))
ax = plt.gca()
plt.title('cold plate temperature ramp (cooling)', fontsize = 14)
plt.xlabel('time (minutes)', fontsize = 14)
plt.ylabel('temperature ($^{\circ}$ C)', fontsize = 14)
ax.set_xticks([0, 4, 8, 12, 16, 20])
plt.plot(t_m, temps, color = 'k')
plt.scatter(t_m, temps, color = 'k')
sns.despine()
plt.show()

t = np.arange(0, 21, 2.5)
t_w = np.array([20, 30])
temps = np.array([20, 0, -2, -4, -6, -8, -10, -12, -14])
temps_w = ([-14, 18])

fig = plt.figure(figsize = (6, 4), dpi = dpi)
ax = plt.gca()
plt.title('cold plate temperature ramp', fontsize = 14)
plt.xlabel('time (minutes)', fontsize = 14)
plt.ylabel('temperature ($^{\circ}$ C)', fontsize = 14)
ax.set_xticks([0, 5, 10, 15, 20, 25, 30])
plt.plot(t, temps, color = 'xkcd:dark slate blue')
plt.scatter(t, temps, color = 'xkcd:dark slate blue')
plt.plot(t_w, temps_w, color = 'xkcd:dark red')
plt.scatter(t_w, temps_w, color = 'xkcd:dark red')
sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'temp_ramp.png'))
plt.show()

t = [0, 2.5]
t2 = list(np.arange(6.25, 26, 3.75))
t.extend(t2)
t_w = np.array([25, 30])
temps = np.array([20, 0, -3, -6, -9, -12, -15, -18])
temps_w = ([-18, 18])

fig = plt.figure(figsize = (6, 4), dpi = dpi)
ax = plt.gca()
plt.title('cold plate temperature ramp (cold room)', fontsize = 14)
plt.xlabel('time (minutes)', fontsize = 14)
plt.ylabel('temperature ($^{\circ}$ C)', fontsize = 14)
ax.set_xticks([0, 5, 10, 15, 20, 25, 30])
plt.plot(t, temps, color = 'xkcd:dark slate blue')
plt.scatter(t, temps, color = 'xkcd:dark slate blue')
plt.plot(t_w, temps_w, color = 'xkcd:dark red')
plt.scatter(t_w, temps_w, color = 'xkcd:dark red')
sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'temp_ramp_cold_room.png'))
plt.show()

In [ ]:
# COOLING AND WARMING
# scatter average velocity for bins of temperatures 
sns.set()
sns.set_style('ticks')
dpi = 600

moving = np.unique(data[~data.exclude].flyid)
lab = np.array(data_scp[np.array(data_scp.cold_room == 'n')].flyid)
cold_room = np.array(data_scp[np.array(data_scp.cold_room == 'y')].flyid)
scp = np.array(data_scp[~data_scp.scp_frame.isnull()].flyid)
cc = np.array(data_scp[~data_scp.cc_frame.isnull()].flyid)
only_scp = np.array(data_scp[np.array(data_scp.cc_frame.isnull()) & np.array(~data_scp.scp_frame.isnull())].flyid)
only_cc = np.array(data_scp[np.array(data_scp.scp_frame.isnull()) & np.array(~data_scp.cc_frame.isnull())].flyid)
nothing = np.array(data_scp[data_scp.scp_frame.isnull() & data_scp.cc_frame.isnull()].flyid)

flyids = np.intersect1d(lab, moving)
flyids = np.intersect1d(flyids, only_cc)
print(flyids)
n = len(flyids)
# cond = 'sponetaneous ice nucleation'
cond = 'chill coma'
subset = v_df[v_df.flyid.isin(flyids)]

fig = plt.figure(figsize = (6, 4), dpi = dpi)
ax = plt.gca()
plt.title(cond + ' (n=' + str(n) + ')', fontsize = 14)
# plt.title('average snow fly velocity', fontsize = 14)
plt.xlabel('cold plate temperature ($^{\circ}$C)', fontsize = 14)
plt.ylabel('average snow fly speed (mm/s)', fontsize = 14)    
    
avg_velocities_c = np.nanmean(np.vstack(subset.velocity_cooling), axis = 0)
avg_velocities_w = np.nanmean(np.vstack(subset.velocity_warming), axis = 0)
avg_temps = bins[:-1] + delta/2
stderr_c = stats.sem(np.vstack(subset.velocity_cooling), axis = 0, nan_policy = 'omit')
stderr_w = stats.sem(np.vstack(subset.velocity_warming), axis = 0, nan_policy = 'omit')
std_c = np.nanstd(np.vstack(subset.velocity_cooling), axis = 0)
std_w = np.nanstd(np.vstack(subset.velocity_warming), axis = 0)

plt.plot(avg_temps[np.isfinite(avg_velocities_c)], avg_velocities_c[np.isfinite(avg_velocities_c)], color = 'xkcd:dark slate blue', markersize = 5, marker = 'o', label = 'cooling')
plt.plot(avg_temps[np.isfinite(avg_velocities_w)], avg_velocities_w[np.isfinite(avg_velocities_w)], color = 'xkcd:dark red', markersize = 5, marker = 'o', label = 'warming')
plt.fill_between(avg_temps, avg_velocities_c - std_c, avg_velocities_c + std_c, color = 'xkcd:dark slate blue', alpha = 0.3, linewidth = 0)
plt.fill_between(avg_temps, avg_velocities_w - std_w, avg_velocities_w + std_w, color = 'xkcd:dark red', alpha = 0.3, linewidth = 0)
# plt.errorbar(avg_temps, avg_velocities_c, yerr = errors_c, color = 'xkcd:cyan', linewidth = 1, capsize = 2, markersize = 5, marker = 'o')
# plt.errorbar(avg_temps, avg_velocities_w, yerr = errors_w, color = 'xkcd:salmon', linewidth = 1, capsize = 2, markersize = 5, marker = 'o')
hs, ls = ax.get_legend_handles_labels()
plt.axhline(y = 0, color = 'k', linewidth = 1, linestyle = ':')
# plt.ylim(bottom = 0)
plt.xlim([-11, 18])
plt.ylim([-1, 5])
sns.despine()
plt.tight_layout()
# plt.savefig(os.path.join(savepath, 'scp_velocity_temp_thresh.png'))
# plt.savefig(os.path.join(savepath, 'cc_velocity_temp_thresh.png'))
plt.show()

fig = plt.figure(figsize = (3, 3), dpi = dpi)
plt.legend(labels = ls, handles = hs, fontsize = 14)
plt.axis('off')
# plt.savefig(os.path.join(savepath, 'velocity_temp_legend.png'))
plt.show()

In [ ]:
# calculate average acceleration for bins of temperature (all flies)
flyids = np.unique(data.flyid)
max_frames = 54000
window_size = 151
delta = 0.5
bins = np.arange(-18, 18, delta)
a_df = pd.DataFrame()
a_thresh = 0

for i, fly in enumerate(flyids):
    
    fly_data = data[data.flyid == fly]
    
    ix = np.argmin(fly_data.cold_plate_temp_filtered)
    abs_acceleration = np.abs(fly_data.acceleration)
    acceleration_c = np.array(abs_acceleration[:ix])
    temperatures_c = np.array(fly_data.cold_plate_temp_filtered[:ix])
    acceleration_w = np.array(abs_acceleration[ix:])
    temperatures_w = np.array(fly_data.cold_plate_temp_filtered[ix:])
    
    mask_c = acceleration_c >= a_thresh
    mask_w = acceleration_w >= a_thresh
    acceleration_c = acceleration_c[mask_c]
    acceleration_w = acceleration_w[mask_w]
    temperatures_c = temperatures_c[mask_c]
    temperatures_w = temperatures_w[mask_w]

    a_c = np.zeros([len(bins)-1])
    a_w = np.zeros([len(bins)-1])
    
    for j in range(len(bins)-1): 
        temp_mask_c = (temperatures_c >= bins[j]) & (temperatures_c < bins[j+1])
        a_c[j] = np.nanmean(acceleration_c[temp_mask_c])
        temp_mask_w = (temperatures_w >= bins[j]) & (temperatures_w < bins[j+1])
        a_w[j] = np.nanmean(acceleration_w[temp_mask_w])
        
    row = {'flyid': fly, 'acceleration_cooling': a_c, 'acceleration_warming': a_w}
    a_df = a_df.append(row, ignore_index = True)

In [ ]:
# COOLING AND WARMING
# scatter average acceleration for bins of temperatures 
sns.set()
sns.set_style('ticks')

moving = np.unique(data[~data.exclude].flyid)
lab = np.array(data_scp[np.array(data_scp.cold_room == 'n')].flyid)
cold_room = np.array(data_scp[np.array(data_scp.cold_room == 'y')].flyid)
scp = np.array(data_scp[~data_scp.scp_frame.isnull()].flyid)
cc = np.array(data_scp[~data_scp.cc_frame.isnull()].flyid)
only_scp = np.array(data_scp[np.array(data_scp.cc_frame.isnull()) & np.array(~data_scp.scp_frame.isnull())].flyid)
only_cc = np.array(data_scp[np.array(data_scp.scp_frame.isnull()) & np.array(~data_scp.cc_frame.isnull())].flyid)
nothing = np.array(data_scp[data_scp.scp_frame.isnull() & data_scp.cc_frame.isnull()].flyid)

flyids = np.intersect1d(lab, moving)
flyids = np.intersect1d(flyids, only_scp)
print(flyids)
n = len(flyids)
cond = 'lab, only scp'
subset = a_df[a_df.flyid.isin(flyids)]

fig = plt.figure(figsize = (8, 4))
ax = plt.gca()
plt.title(cond + ' (n=' + str(n) + ')', fontsize = 14)
# plt.title('average snow fly acceleration', fontsize = 14)
plt.xlabel('cold plate temperature ($^{\circ}$C)', fontsize = 14)
plt.ylabel('absolute acceleration (mm/s$^{2}$)', fontsize = 14)       

avg_acceleration_c = np.nanmean(np.vstack(subset.acceleration_cooling), axis = 0)
avg_acceleration_w = np.nanmean(np.vstack(subset.acceleration_warming), axis = 0)
avg_temps = bins[:-1] + delta/2
errors_c = stats.sem(np.vstack(subset.acceleration_cooling), axis = 0, nan_policy = 'omit')
errors_w = stats.sem(np.vstack(subset.acceleration_warming), axis = 0, nan_policy = 'omit')
std_c = np.nanstd(np.vstack(subset.acceleration_cooling), axis = 0)
std_w = np.nanstd(np.vstack(subset.acceleration_warming), axis = 0)

plt.plot(avg_temps[np.isfinite(avg_acceleration_c)], avg_acceleration_c[np.isfinite(avg_acceleration_c)], color = 'xkcd:dark slate blue', markersize = 5, marker = 'o', label = 'cooling')
plt.plot(avg_temps[np.isfinite(avg_acceleration_w)], avg_acceleration_w[np.isfinite(avg_acceleration_w)], color = 'xkcd:dark red', markersize = 5, marker = 'o', label = 'warming')
plt.fill_between(avg_temps, avg_acceleration_c - std_c, avg_acceleration_c + std_c, color = 'xkcd:dark slate blue', alpha = 0.3, linewidth = 0)
plt.fill_between(avg_temps, avg_acceleration_w - std_w, avg_acceleration_w + std_w, color = 'xkcd:dark red', alpha = 0.3, linewidth = 0)
# plt.errorbar(avg_temps, avg_acceleration_c, yerr = errors_c, color = 'xkcd:dark slate blue', linewidth = 1, capsize = 2, markersize = 5, marker = 'o')
# plt.errorbar(avg_temps, avg_acceleration_w, yerr = errors_w, color = 'xkcd:dark red', linewidth = 1, capsize = 2, markersize = 5, marker = 'o')
hs, ls = ax.get_legend_handles_labels()
plt.xlim(left = -18)
plt.axhline(y = 0, color = 'k', linewidth = 1, linestyle = ':') 
sns.despine()
plt.show()
    
fig = plt.figure(figsize = (3, 3))
plt.legend(labels = ls, handles = hs, fontsize = 14)
plt.axis('off')
plt.show()

In [ ]:
# DURING COOLING AND REWARMING
# plot velocity vs temperature, but using a moving average for velocity and temperature 
flyids = np.unique(data.flyid)
window_size = 500

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    
    ix = np.argmin(fly_data.cold_plate_temp_filtered)
    abs_velocity = np.abs(fly_data.velocity_filtered)
    velocities_c = abs_velocity[:ix]
    temperatures_c = fly_data.cold_plate_temp_filtered[:ix]
    velocities_w = abs_velocity[ix:]
    temperatures_w = fly_data.cold_plate_temp_filtered[ix:]
    
    velocity_moving_average_c = np.convolve(velocities_c, np.ones(window_size)/window_size, mode='valid')
    temp_moving_average_c = np.convolve(temperatures_c, np.ones(window_size)/window_size, mode='valid')
    velocity_moving_average_w = np.convolve(velocities_w, np.ones(window_size)/window_size, mode='valid')
    temp_moving_average_w = np.convolve(temperatures_w, np.ones(window_size)/window_size, mode='valid')
    # moving = velocities > 20
    
    fig = plt.figure(figsize = (8, 4))
    plt.title(fly, fontsize = 14)
    plt.xlabel('temperature ($^{\circ}$C)', fontsize = 14)
    plt.ylabel('absolute velocity (mm/s)', fontsize = 14)
    plt.scatter(temp_moving_average_c, velocity_moving_average_c, color = 'xkcd:light blue', s = 1, marker = 'o', alpha = 0.9, label = 'cooling')
    plt.scatter(temp_moving_average_w, velocity_moving_average_w, color = 'xkcd:salmon', s = 1, marker = 'o', alpha = 0.7, label = 'heating')
    
    # plt.legend(fontsize = 12)
    sns.despine()
    plt.show()

In [ ]:
# plot velocity vs temperature 
flyids = np.unique(data.flyid)
movement_thresh = 0.1

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    
    ix = np.argmin(fly_data.cold_plate_temp_filtered)
    abs_velocity = np.abs(fly_data.velocity_filtered)
    velocities = abs_velocity[:ix]
    temperatures = fly_data.cold_plate_temp_filtered[:ix]
    
    moving = velocities > movement_thresh
    
    fig = plt.figure()
    plt.xlabel('temperature ($^{\circ}$C)', fontsize = 14)
    plt.ylabel('absolute velocity (a.u.)', fontsize = 14)
    plt.title(fly, fontsize = 14)
    plt.scatter(temperatures[moving], velocities[moving], color = 'k', s = 2, marker = 'o')
    sns.despine()
    plt.show()

In [ ]:
# for each fly, plot the interpolated average temperature, max temperature, and cold plate temperature 
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
max_frames = 54000

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (10, 6))
    plt.title(fly, fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('filtered temperature ($^{\circ}$C)', fontsize = 14)
    
    plt.plot(t_m, fly_data.max_temp_interp[:len(t_m)], 'xkcd:light blue', linewidth = 0.5, label = 'max temp')
    plt.plot(t_m, fly_data.avg_temp_interp[:len(t_m)], 'xkcd:slate blue', linewidth = 0.5, label = 'avg temp')
    plt.plot(t_m, fly_data.cold_plate_temp_interp[:len(t_m)], 'xkcd:black', linewidth = 0.5, label = 'cold plate temp')
    
    sns.despine()
    plt.legend(fontsize = 12)
    plt.show()

In [ ]:
# for each fly, plot the filtered average temperature, max temperature, and cold plate temperature 
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
max_frames = 54000

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (10, 6))
    plt.title(fly, fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('filtered temperature ($^{\circ}$C)', fontsize = 14)
    
    plt.plot(t_m, fly_data.max_temp_filtered[:len(t_m)], 'xkcd:light blue', linewidth = 1, label = 'max temp')
    plt.plot(t_m, fly_data.avg_temp_filtered[:len(t_m)], 'xkcd:slate blue', linewidth = 1, label = 'avg temp')
    plt.plot(t_m, fly_data.cold_plate_temp_filtered[:len(t_m)], 'xkcd:black', linewidth = 1, label = 'cold plate temp')
    
    sns.despine()
    plt.legend(fontsize = 12)
    plt.show()

In [ ]:
# EVERY 60th FRAME
# for each fly, plot the filtered average temperature, max temperature, and cold plate temperature 
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
n = 60
max_frames = 54000

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (10, 6))
    ax = plt.gca()
    plt.title(fly)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('filtered temperature ($^{\circ}$C)', fontsize = 14)
    
    t_m = t_m[::n]
    max_temps = fly_data.max_temp_filtered[::n]
    avg_temps = fly_data.avg_temp_filtered[::n]
    cp_temps = fly_data.cold_plate_temp_filtered[::n]
    
    plt.plot(t_m, max_temps[:len(t_m)], 'xkcd:light blue', linewidth = 1, label = 'max temp')
    plt.plot(t_m, avg_temps[:len(t_m)], 'xkcd:slate blue', linewidth = 1, label = 'avg temp')
    plt.plot(t_m, cp_temps[:len(t_m)], 'xkcd:black', linewidth = 1, label = 'cold plate temp')
    
    hs, ls = ax.get_legend_handles_labels()
    sns.despine()
    plt.show()
    
fig = plt.figure()
plt.legend(labels = ls, handles = hs, fontsize = 12)
plt.show()

In [ ]:
# for each fly, plot the average temperature, max temperature, and cold plate temperature 
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
max_frames = 54000

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (10, 6))
    plt.title(fly)
    plt.xlabel('time (minutes)')
    plt.ylabel('temperature ($^{\circ}$C)')
    
    plt.plot(t_m, fly_data.max_temp[:len(t_m)], 'xkcd:light blue', linewidth = 0.5, label = 'max temp')
    plt.plot(t_m, fly_data.avg_temp[:len(t_m)], 'xkcd:slate blue', linewidth = 0.5, label = 'avg temp')
    plt.plot(t_m, fly_data.cold_plate_temp[:len(t_m)], 'xkcd:black', linewidth = 0.5, label = 'cold plate temp')
    
    sns.despine()
    plt.legend()
    plt.show()

In [ ]:
# for each fly, plot the x position and y position across time 
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
# flies = np.unique(data[data.exclude].flyid)

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    
    fig = plt.figure(figsize = (10, 6))
    ax = plt.gca()
    plt.title(fly)
    plt.xlabel('time (minutes)')
    plt.ylabel('x position (pixels)')   
    # plt.plot(t_m, fly_data.x[:len(t_m)], 'r', linewidth = 0.5, label = 'unfiltered')
    plt.plot(t_m, fly_data.x_interp[:len(t_m)], 'k', linewidth = 0.5, label = 'filtered')
    sns.despine()
    hs, ls = ax.get_legend_handles_labels()
    plt.show()
    
    fig = plt.figure(figsize = (10, 6))
    plt.title(fly)
    plt.xlabel('time (minutes)')
    plt.ylabel('y position (pixels)')   
    # plt.plot(t_m, fly_data.y[:len(t_m)], 'r', linewidth = 0.5, label = 'unfiltered')
    plt.plot(t_m, fly_data.y_interp[:len(t_m)], 'k', linewidth = 0.5, label = 'filtered')
    sns.despine()
    plt.show()
    
fig = plt.figure(figsize = (3,3))
plt.legend(handles = hs, labels = ls)
plt.axis('off')
plt.show()

In [ ]:
# DURING COOLING
# for each fly, plot the position of the fly in the metal ring
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
max_frames = 54000

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    ix = np.argmin(fly_data.cold_plate_temp_filtered)
    
    trial_path = os.path.dirname(fly_data.fullfile.iloc[0])
    # mask_path = os.path.join(path, trial_path, 'mask - Ellipse 1.bmp')
    # mask = plt.imread(mask_path)
    
    fig = plt.figure()
    ax = plt.gca()
    plt.title(fly, fontsize = 14)
    #plt.xlabel('x position (pixels)')
    #plt.ylabel('y position (pixels)')
    # ax.imshow(mask)
    xs = fly_data.x_interp[:ix]
    ys = fly_data.y_interp[:ix]
    ax.plot(ys, xs, 'k', linewidth = 0.5)  
    ax.axis('equal')
    sns.despine()
    ax.invert_yaxis()
    plt.axis('off')
    plt.show()

In [ ]:
# DURING HEATING
# for each fly, plot the position of the fly in the metal ring
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
max_frames = 54000

for fly in flies:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    ix = np.argmin(fly_data.cold_plate_temp_filtered)
    
    trial_path = os.path.dirname(fly_data.fullfile.iloc[0])
    # mask_path = os.path.join(path, trial_path, 'mask - Ellipse 1.bmp')
    # mask = plt.imread(mask_path)
    
    fig = plt.figure()
    ax = plt.gca()
    plt.title(fly, fontsize = 14)
    xs = fly_data.x_interp[ix:]
    ys = fly_data.y_interp[ix:]
    ax.plot(ys, xs, 'k', linewidth = 0.5)    
    ax.axis('equal')
    sns.despine()
    ax.invert_yaxis()
    plt.axis('off')
    plt.show()

In [ ]:
# calculate average proportion of time spent moving for
# bins of temperature (all flies)
flyids = np.unique(data.flyid)
max_frames = 54000
delta = 0.5
bins = np.arange(-12, 18, delta)
prop_df = pd.DataFrame()
v_thresh = 0.1

for i, fly in enumerate(flyids):
    
    fly_data = data[data.flyid == fly]
    
    ix = np.argmin(fly_data.cold_plate_temp_filtered)
    abs_velocity = np.abs(fly_data.velocity)
    velocities_c = np.array(abs_velocity[:ix])
    temperatures_c = np.array(fly_data.cold_plate_temp_filtered[:ix])
    velocities_w = np.array(abs_velocity[ix:])
    temperatures_w = np.array(fly_data.cold_plate_temp_filtered[ix:])
    
    mask_c = velocities_c > v_thresh
    mask_w = velocities_w > v_thresh

    prop_c = np.zeros([len(bins)-1])
    prop_w = np.zeros([len(bins)-1])
    
    for j in range(len(bins)-1): 
        temp_mask_c = (temperatures_c >= bins[j]) & (temperatures_c < bins[j+1])
        prop_c[j] = np.sum(mask_c[temp_mask_c]) / len(velocities_c[temp_mask_c])
        temp_mask_w = (temperatures_w >= bins[j]) & (temperatures_w < bins[j+1])
        prop_w[j] = np.sum(mask_w[temp_mask_w]) / len(velocities_w[temp_mask_w])
    
    row = {'flyid': fly, 'prop_cooling': prop_c, 'prop_warming': prop_w}
    prop_df = prop_df.append(row, ignore_index = True)

In [ ]:
# COOLING AND WARMING
# scatter average proportion of time spent moving for bins of cold plate temperatures 
sns.set()
sns.set_style('ticks')

moving = np.unique(data[~data.exclude].flyid)
lab = np.array(data_scp[np.array(data_scp.cold_room == 'n')].flyid)
cold_room = np.array(data_scp[np.array(data_scp.cold_room == 'y')].flyid)
scp = np.array(data_scp[~data_scp.scp_frame.isnull()].flyid)
cc = np.array(data_scp[~data_scp.cc_frame.isnull()].flyid)
only_scp = np.array(data_scp[np.array(data_scp.cc_frame.isnull()) & np.array(~data_scp.scp_frame.isnull())].flyid)
only_cc = np.array(data_scp[np.array(data_scp.scp_frame.isnull()) & np.array(~data_scp.cc_frame.isnull())].flyid)

flyids = np.intersect1d(lab, moving)
flyids = np.intersect1d(flyids, only_scp)
print(flyids)
n = len(flyids)
cond = 'spontaneous ice nucleation'
# cond = 'chill coma'
subset = prop_df[prop_df.flyid.isin(flyids)]

fig = plt.figure(figsize = (6, 4), dpi = dpi)
ax = plt.gca()
plt.title(cond + ' (n=' + str(n) + ')', fontsize = 14)
# plt.title('average proportion of time spent moving at different temperatures', fontsize = 14)
plt.xlabel('cold plate temperature ($^{\circ}$C)', fontsize = 14)
plt.ylabel('proportion of time spent moving', fontsize = 14)    

avg_prop_c = np.nanmean(np.vstack(subset.prop_cooling), axis = 0)
avg_prop_w = np.nanmean(np.vstack(subset.prop_warming), axis = 0)
avg_temps = bins[:-1] + delta/2
stderr_c = stats.sem(np.vstack(subset.prop_cooling), axis = 0, nan_policy = 'omit')
stderr_w = stats.sem(np.vstack(subset.prop_warming), axis = 0, nan_policy = 'omit')
std_c = np.nanstd(np.vstack(subset.prop_cooling), axis = 0)
std_w = np.nanstd(np.vstack(subset.prop_warming), axis = 0)
plt.plot(avg_temps[np.isfinite(avg_prop_c)], avg_prop_c[np.isfinite(avg_prop_c)], color = 'xkcd:dark slate blue', markersize = 5, marker = 'o', label = 'cooling')
plt.plot(avg_temps[np.isfinite(avg_prop_w)], avg_prop_w[np.isfinite(avg_prop_w)], color = 'xkcd:dark red', markersize = 5, marker = 'o', label = 'warming')
plt.fill_between(avg_temps, avg_prop_c - std_c, avg_prop_c + std_c, color = 'xkcd:dark slate blue', alpha = 0.3, linewidth = 0)
plt.fill_between(avg_temps, avg_prop_w - std_w, avg_prop_w + std_w, color = 'xkcd:dark red', alpha = 0.3, linewidth = 0)
# plt.errorbar(avg_temps, avg_prop_c, yerr = stderr_c, color = 'xkcd:dark slate blue', linewidth = 1, capsize = 2, markersize = 5, marker = 'o')
# plt.errorbar(avg_temps, avg_prop_w, yerr = stderr_w, color = 'xkcd:dark red', linewidth = 1, capsize = 2, markersize = 5, marker = 'o')
hs, ls = ax.get_legend_handles_labels()
plt.axhline(y = 0, color = 'k', linewidth = 1, linestyle = ':') 
plt.axhline(y = 1, color = 'k', linewidth = 1, linestyle = ':') 
plt.ylim([-0.3, 1.3]) 
plt.xlim([-12, 18])
sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'scp_velocity_temp_prop01.png'))
# plt.savefig(os.path.join(savepath, 'cc_velocity_temp_prop01.png'))
plt.show()

fig = plt.figure(figsize = (3, 3), dpi = dpi)
plt.legend(labels = ls, handles = hs, fontsize = 14)
plt.axis('off')
plt.savefig(os.path.join(savepath, 'velocity_temp_prop_legend.png'))
plt.show()

In [ ]:
# difference between the average fly temperature and the cold plate temperature (filtered)
sns.set()
sns.set_style('ticks')
flyids = np.unique(data.flyid)

for fly in flyids:
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    delta_temp = fly_data.avg_temp_filtered - fly_data.cold_plate_temp_filtered
    delta_temp = delta_temp[:len(t_m)]
    max_val = np.max(abs(delta_temp))
    
    window_size = 501
    running_mean = np.convolve(delta_temp, np.ones(window_size)/window_size, mode='valid')
    
    fig = plt.figure(figsize = (8, 4))
    ax = plt.gca()
    #ax.spines['left'].set_position('zero')
    #ax.spines['bottom'].set_position('zero')
    plt.title(fly, fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('difference in temperature ($^{\circ}$C)', fontsize = 14)  
    
    plt.plot(t_m, delta_temp, 'k', linewidth = 0.5)
    plt.plot(t_m[int(window_size/2):-int(window_size/2)], running_mean, 'xkcd:light blue')
    plt.axhline(y = 0, color = 'k', linewidth = 1, linestyle = ':') 
    ax.set_ylim([-10, 10])
    
    sns.despine()
    plt.show()


In [ ]:
# calculate average deviations between average snow fly temperature
# and cold plate temperature for bins of temperature (all flies)
flyids = np.unique(data.flyid)
max_frames = 54000
window_size = 151
delta = 0.5
bins = np.arange(-18, 18, delta)
dev_df = pd.DataFrame()
v_thresh = 0

for i, fly in enumerate(flyids):
    
    fly_data = data[data.flyid == fly]
    
    ix = np.argmin(fly_data.cold_plate_temp_filtered)
    deviations = fly_data.avg_temp_filtered - fly_data.cold_plate_temp_filtered
    deviations_c = np.array(deviations[:ix])
    temperatures_c = np.array(fly_data.cold_plate_temp_filtered[:ix])
    deviations_w = np.array(deviations[ix:])
    temperatures_w = np.array(fly_data.cold_plate_temp_filtered[ix:])
    
    velocities_c = np.abs(fly_data.velocity.iloc[:ix])
    velocities_w = np.abs(fly_data.velocity.iloc[ix:])
    mask_c = velocities_c >= v_thresh
    mask_w = velocities_w >= v_thresh
    deviations_c = deviations_c[mask_c]
    deviations_w = deviations_w[mask_w]
    temperatures_c = temperatures_c[mask_c]
    temperatures_w = temperatures_w[mask_w]
    
    d_c = np.zeros(len(bins)-1)
    d_w = np.zeros(len(bins)-1)

    for j in range(len(bins)-1): 
        temp_mask_c = (temperatures_c >= bins[j]) & (temperatures_c < bins[j+1])
        d_c[j] = np.nanmean(deviations_c[temp_mask_c])
        temp_mask_w = (temperatures_w >= bins[j]) & (temperatures_w < bins[j+1])
        d_w[j] = np.nanmean(deviations_w[temp_mask_w])
    
    row = {'flyid': fly, 'dev_cooling': d_c, 'dev_warming': d_w}
    dev_df = dev_df.append(row, ignore_index = True)

In [ ]:
# COOLING AND WARMING
# scatter average temperature deviation for bins of cold plate temperatures 
sns.set()
sns.set_style('ticks')

moving = np.unique(data[~data.exclude].flyid)
lab = np.array(data_scp[np.array(data_scp.cold_room == 'n')].flyid)
cold_room = np.array(data_scp[np.array(data_scp.cold_room == 'y')].flyid)
scp = np.array(data_scp[~data_scp.scp_frame.isnull()].flyid)
cc = np.array(data_scp[~data_scp.cc_frame.isnull()].flyid)
only_scp = np.array(data_scp[np.array(data_scp.cc_frame.isnull()) & np.array(~data_scp.scp_frame.isnull())].flyid)
only_cc = np.array(data_scp[np.array(data_scp.scp_frame.isnull()) & np.array(~data_scp.cc_frame.isnull())].flyid)
nothing = np.array(data_scp[data_scp.scp_frame.isnull() & data_scp.cc_frame.isnull()].flyid)

flyids = np.intersect1d(lab, moving)
flyids = np.intersect1d(flyids, only_scp)
print(flyids)
n = len(flyids)
cond = 'lab, spontaneous ice nucleation'
subset = dev_df[dev_df.flyid.isin(flyids)]

fig = plt.figure(figsize = (8, 4))
ax = plt.gca()
plt.title(cond + ' (n=' + str(n) + ')', fontsize = 14)
# plt.title('average deviation of snow fly temperature from cold plate temperature', fontsize = 14)
plt.xlabel('cold plate temperature ($^{\circ}$C)', fontsize = 14)
plt.ylabel('temperature difference ($^{\circ}$C)', fontsize = 14)       

avg_dev_c = np.nanmean(np.vstack(subset.dev_cooling), axis = 0)
avg_dev_w = np.nanmean(np.vstack(subset.dev_warming), axis = 0)
avg_temps = bins[:-1] + delta/2
stderr_c = stats.sem(np.vstack(subset.dev_cooling), axis = 0, nan_policy = 'omit')
stderr_w = stats.sem(np.vstack(subset.dev_warming), axis = 0, nan_policy = 'omit')
std_c = np.nanstd(np.vstack(subset.dev_cooling), axis = 0)
std_w = np.nanstd(np.vstack(subset.dev_warming), axis = 0)
plt.plot(avg_temps[np.isfinite(avg_dev_c)], avg_dev_c[np.isfinite(avg_dev_c)], color = 'xkcd:dark slate blue', markersize = 5, marker = 'o', label = 'cooling')
plt.plot(avg_temps[np.isfinite(avg_dev_w)], avg_dev_w[np.isfinite(avg_dev_w)], color = 'xkcd:dark red', markersize = 5, marker = 'o', label = 'warming')
plt.fill_between(avg_temps, avg_dev_c - std_c, avg_dev_c + std_c, color = 'xkcd:dark slate blue', alpha = 0.3, linewidth = 0)
plt.fill_between(avg_temps, avg_dev_w - std_w, avg_dev_w + std_w, color = 'xkcd:dark red', alpha = 0.3, linewidth = 0)
# plt.errorbar(avg_temps, avg_dev_c, yerr = stderr_c, color = 'xkcd:dark slate blue', linewidth = 1, capsize = 2, markersize = 5, marker = 'o')
# plt.errorbar(avg_temps, avg_dev_w, yerr = stderr_w, color = 'xkcd:dark red', linewidth = 1, capsize = 2, markersize = 5, marker = 'o')
hs, ls = ax.get_legend_handles_labels()
plt.axhline(y = 0, color = 'k', linewidth = 1, linestyle = ':')
plt.xlim(left = -11)
plt.ylim([-7, 5.5])
sns.despine()
plt.show()

fig = plt.figure(figsize = (3, 3))
plt.legend(labels = ls, handles = hs, fontsize = 14)
plt.axis('off')
plt.show()

In [ ]:
# one plot for all flies - lab
# difference between the average fly temperature and the cold plate temperature (filtered)
sns.set()
sns.set_style('ticks')
window_size = 501
max_frames = 54000
colors = sns.color_palette("husl", 8)

moving = np.unique(data[~data.exclude].flyid)
lab = np.array(data_scp[np.array(data_scp.cold_room == 'n')].flyid)
cold_room = np.array(data_scp[np.array(data_scp.cold_room == 'y')].flyid)
scp = np.array(data_scp[~data_scp.scp_frame.isnull()].flyid)
cc = np.array(data_scp[~data_scp.cc_frame.isnull()].flyid)
only_scp = np.array(data_scp[np.array(data_scp.cc_frame.isnull()) & np.array(~data_scp.scp_frame.isnull())].flyid)
only_cc = np.array(data_scp[np.array(data_scp.scp_frame.isnull()) & np.array(~data_scp.cc_frame.isnull())].flyid)
nothing = np.array(data_scp[data_scp.scp_frame.isnull() & data_scp.cc_frame.isnull()].flyid)

flyids = np.intersect1d(lab, moving)
flyids = np.intersect1d(flyids, only_scp)

# flyids = np.unique(data_scp[data_scp.cold_room == 'n'].flyid)
fig = plt.figure(figsize = (8, 6))
ax = plt.gca()
ax.set_ylim([-5, 6])
plt.title('deviation of snow fly temperature from cold plate temperature (lab)', fontsize = 14)
ax.set_xlabel('time (minutes)', fontsize = 14)
ax.set_ylabel('temperature difference ($^{\circ}$C)', fontsize = 14) 

plt.axhline(y = 0, color = 'k', linewidth = 1, linestyle = ':')
cp_traces = np.empty((max_frames, len(flyids)))

for j, fly in enumerate(flyids):
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    delta_temp = fly_data.avg_temp_filtered - fly_data.cold_plate_temp_filtered
    delta_temp = delta_temp[:len(t_m)]
    
    running_mean = np.convolve(delta_temp, np.ones(window_size)/window_size, mode='valid')  
    cp_traces[:len(t_m), j] = fly_data.cold_plate_temp_filtered[:len(t_m)] 
    # plt.plot(t_m, delta_temp, 'k', linewidth = 0.5)
    ax.plot(t_m[int(window_size/2):-int(window_size/2)], running_mean, color = colors[j%8], linewidth = 0.5)
    
ax2 = ax.twinx()
ax2.set_ylabel('cold plate temperature ($^{\circ}$C)', fontsize = 14)
avg_cp_trace = np.nanmean(cp_traces, axis = 1)
ax2.plot(t_m, avg_cp_trace, color = 'k', linewidth = 0.5, label = 'cold plate') 

sns.despine()
ax2.spines['right'].set_visible(True)
plt.show()


In [ ]:
# one plot for all flies - cold room
# difference between the average fly temperature and the cold plate temperature (filtered)
sns.set()
sns.set_style('ticks')
window_size = 501
max_frames = 53900
colors = sns.color_palette("husl", 8)

flyids = np.unique(data_scp[data_scp.cold_room == 'y'].flyid)
fig = plt.figure(figsize = (8, 6))
ax = plt.gca()
ax.set_ylim([-10, 10])
plt.title('deviation of snow fly temperature from cold plate temperature (cold room)', fontsize = 14)
ax.set_xlabel('time (minutes)', fontsize = 14)
ax.set_ylabel('temperature difference ($^{\circ}$C)', fontsize = 14) 

plt.axhline(y = 0, color = 'k', linewidth = 1, linestyle = ':')
cp_traces = np.empty((max_frames, len(flyids))) 

for j, fly in enumerate(flyids):
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    delta_temp = fly_data.avg_temp_filtered - fly_data.cold_plate_temp_filtered
    delta_temp = delta_temp[:len(t_m)]
    
    running_mean = np.convolve(delta_temp, np.ones(window_size)/window_size, mode='valid') 
    # plt.plot(t_m, delta_temp, 'k', linewidth = 0.5)
    plt.plot(t_m[int(window_size/2):-int(window_size/2)], running_mean, color = colors[j%8], linewidth = 0.5)
    cp_traces[:len(t_m), j] = fly_data.cold_plate_temp_filtered[:len(t_m)] 

ax2 = ax.twinx()
ax2.set_ylabel('cold plate temperature ($^{\circ}$C)', fontsize = 14)
avg_cp_trace = np.nanmean(cp_traces, axis = 1)
ax2.plot(t_m, avg_cp_trace, color = 'k', linewidth = 0.5, label = 'cold plate') 

sns.despine()
ax2.spines['right'].set_visible(True)
plt.show()


In [ ]:
# cold plate temperature for all trials - how consistent is the ramp? 
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
colors = sns.color_palette('mako', len(flies))

fig = plt.figure(figsize = (10, 6))
plt.title('cold plate temperatures')
plt.xlabel('time (minutes)')
plt.ylabel('temperature ($^{\circ}$C)')

for j, fly in enumerate(flies):
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    plt.plot(t_m, fly_data.cold_plate_temp[:len(t_m)], color = colors[j], linewidth = 0.5)
    
sns.despine()
plt.show()


In [ ]:
# snow fly temperatures for all trials
sns.set()
sns.set_style('ticks')
flies = np.unique(data.flyid)
colors = sns.color_palette('mako', len(flies))

fig = plt.figure(figsize = (10, 6))
ax = plt.gca()
plt.title('avg snowfly temperatures')
plt.xlabel('time (minutes)')
plt.ylabel('temperature ($^{\circ}C$)')

for j, fly in enumerate(flies):
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)
    plt.plot(t_m, fly_data.avg_temp[:len(t_m)], color = colors[j], linewidth = 0.5, label = fly)

hs, ls = ax.get_legend_handles_labels()
sns.despine()
plt.show()

fig = plt.figure(figsize = (3,3))
plt.axis('off')
plt.legend(handles = hs, labels = ls)
sns.despine()
plt.show()


In [ ]:
# plot the pixel intensity across time for each fly
sns.set()
sns.set_style('ticks')
fs = 14
flies = np.unique(data.flyid)
colors = sns.color_palette('mako', 3)

for j, fly in enumerate(flies):
    
    fig = plt.figure(figsize = (8, 5))
    ax = plt.gca()
    plt.title(fly, fontsize = fs)
    ax.set_xlabel('time (minutes)', fontsize = fs)
    ax.set_ylabel('temperature ($^{\circ}$C)', fontsize = fs)
    
    fly_data = data[data.flyid == fly]
    cp_temp = np.array(fly_data.cold_plate_temp[:len(t_m)])
    cp_temp[cp_temp == 0] = np.nan 
    fly_temp = np.array(fly_data.avg_temp[:len(t_m)])
    fly_temp[fly_temp == 0] = np.nan 
    t_s, t_m = get_video_length(len(fly_data), thresh = max_frames)    
    ax.plot(t_m, cp_temp, color = colors[0], linewidth = 0.5, label = 'average cold plate temp')
    ax.plot(t_m, fly_temp, color = colors[1], linewidth = 0.5, label = 'average snow fly temp')

    ax2 = ax.twinx()
    ax2.set_ylabel('movement (norm.)', fontsize = fs)
    movement_norm = fly_data.movement[:len(t_m)] / max(fly_data.movement[:len(t_m)])
    movement_norm[:1000] = 0
    ax2.plot(t_m, movement_norm, color = colors[2], linewidth = 0.5, label = 'activity', alpha= 0.8)
    
    hs, ls = ax.get_legend_handles_labels()
    sns.despine(right = False)
    plt.show()

fig = plt.figure(figsize = (3,3))
plt.axis('off')
plt.legend(handles = hs, labels = ls, fontsize = fs)
plt.show()


In [ ]:
path_scp = r'G:\My Drive\Tuthill Lab Shared\Katie\thermal_experiments\data\cold_tolerance_summary - snow flies.csv'
data_scp = pd.read_csv(path_scp)
data_scp = data_scp[data_scp['scp_germanium_temp'].isnull()]
exclude_dates = ['10.30.20', '11.6.20', '11.13.20', '11.20.20', '12.4.20', '12.11.20']
exclude_flies = [] # ['SF0063_1', 'SF0115_1']  ['SF0080_1', 'SF0086_1']
data_scp = data_scp[~data_scp['trial'].isnull()]
data_scp['trial'] = data_scp['trial'].astype(int)
data_scp['flyid'] = data_scp['fly_id'] + '_' + data_scp['trial'].astype(str)
data_scp = data_scp[~data_scp.date.isin(exclude_dates)]
data_scp = data_scp[~data_scp.flyid.isin(exclude_flies)]
data_scp = data_scp[~data_scp.trial.isnull()]

In [ ]:
scp_ambient = data_scp[~data_scp['scp_ambient_temp'].isnull()]['scp_ambient_temp'].astype(float)
scp_fly_pre = data_scp[~data_scp['scp_fly_temp_pre'].isnull()]['scp_fly_temp_pre'].astype(float)
scp_fly_post = data_scp[~data_scp['scp_fly_temp_post'].isnull()]['scp_fly_temp_post'].astype(float)

cc_ambient = data_scp[~data_scp['cc_ambient_temp'].isnull()]['cc_ambient_temp']
cc_fly = data_scp[~data_scp['cc_fly_temp'].isnull()]['cc_fly_temp']
recovery_ambient = data_scp[~data_scp['recovery_ambient_temp'].isnull()]['recovery_ambient_temp']
recovery_fly = data_scp[~data_scp['recovery_fly_temp'].isnull()]['recovery_fly_temp']

In [ ]:
# histograms of chill coma and supercooling 
sns.set()
sns.set_style('ticks')

cr_data = data_scp[data_scp.cold_room == 'y']
lab_data = data_scp[data_scp.cold_room == 'n']
scp_fly_cold_room = cr_data[~cr_data['scp_fly_temp_pre'].isnull()]['scp_fly_temp_pre'].astype(float)
scp_fly_lab = lab_data[~lab_data['scp_fly_temp_pre'].isnull()]['scp_fly_temp_pre'].astype(float)
cc_fly_lab = lab_data[~lab_data['cc_fly_temp'].isnull()]['cc_fly_temp']

figure = plt.figure(figsize = (6, 4), dpi = 600)
plt.title('distribution of supercooling points', fontsize = 14)
plt.ylabel('snow fly count', fontsize = 14)
plt.xlabel('fly temperature ($^{\circ}$C)', fontsize = 14)
plt.xlim([-18, -4])
plt.ylim([0, 20])

cr_color = '#665fd1'
lab_color = '#048243'
edges = np.arange(-18, -4, 1)
plt.hist(scp_fly_cold_room, bins = edges, color = cr_color, alpha = 0.5, label = 'cold room (n = ' + str(len(scp_fly_cold_room)) + ')')
plt.hist(scp_fly_lab, bins = edges, color = lab_color, alpha = 0.5, label = 'lab (n = ' + str(len(scp_fly_lab)) + ')')
# plt.hist(cc_fly_lab, bins = edges, color = 'b', alpha = 0.5, label = 'CT$_{min}$, lab (n = ' + str(len(cc_fly)) + ')')

ax = plt.gca()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_yticks(ticks = np.arange(0, 21, 4))
hs, ls = ax.get_legend_handles_labels()
# plt.legend(fontsize = 12, loc = 'upper right')
sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'scp_cold_room_vs_lab_a.png'))
plt.show()

fig = plt.figure(figsize = (3,3), dpi = 600)
plt.legend(handles = hs, labels = ls, fontsize = 12)
plt.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'scp_cold_room_vs_lab_legend.png'))
plt.show()

In [ ]:
# histograms of supercooling - cold room vs lab
sns.set()
sns.set_style('ticks')

cr_data = data_scp[data_scp.cold_room == 'y']
lab_data = data_scp[data_scp.cold_room == 'n']
scp_fly_cold_room = cr_data[~cr_data['scp_ambient_temp'].isnull()]['scp_ambient_temp'].astype(float)
scp_fly_lab = lab_data[~lab_data['scp_ambient_temp'].isnull()]['scp_ambient_temp'].astype(float)
cc_fly_lab = lab_data[~lab_data['cc_ambient_temp'].isnull()]['cc_ambient_temp']

figure = plt.figure(figsize = (6, 4), dpi = 600)
plt.title('distribution of supercooling points', fontsize = 14)
plt.ylabel('snow fly count', fontsize = 14)
plt.xlabel('cold plate temperature ($^{\circ}$C)', fontsize = 14)
plt.xlim([-18, -4])
plt.ylim([0, 20])
cr_color = '#665fd1'
lab_color = '#048243'
edges = np.arange(-18, -4, 1)
plt.hist(scp_fly_cold_room, bins = edges, color = cr_color, alpha = 0.5, label = 'cold room (n = ' + str(len(scp_fly_cold_room)) + ')')
plt.hist(scp_fly_lab, bins = edges, color = lab_color, alpha = 0.5, label = 'lab (n = ' + str(len(scp_fly_lab)) + ')')
# plt.hist(cc_fly_lab, bins = edges, color = 'b', alpha = 0.5, label = 'CT$_{min}$, lab (n = ' + str(len(cc_fly)) + ')')

ax = plt.gca()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_yticks(ticks = np.arange(0, 21, 4))

# plt.legend(fontsize = 12, loc = 'upper left')
sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'scp_cold_room_vs_lab_b.png'))
plt.show()


In [ ]:
# histograms of supercooling and chill coma 
sns.set()
sns.set_style('ticks')

cr_data = data_scp[data_scp.cold_room == 'y']
lab_data = data_scp[data_scp.cold_room == 'n']
scp_fly_cold_room = cr_data[~cr_data['scp_fly_temp_pre'].isnull()]['scp_fly_temp_pre'].astype(float)
scp_fly_lab = lab_data[~lab_data['scp_fly_temp_pre'].isnull()]['scp_fly_temp_pre'].astype(float)
cc_fly_lab = lab_data[~lab_data['cc_fly_temp'].isnull()]['cc_fly_temp']

figure = plt.figure(figsize = (6, 4), dpi = 600)
plt.title('distribution of cold tolerance thresholds', fontsize = 14)
plt.ylabel('snow fly count', fontsize = 14)
plt.xlabel('snow fly temperature ($^{\circ}$C)', fontsize = 14)
plt.xlim([-12, -4])
plt.ylim([0, 16])
cr_color = '#665fd1'
lab_color = '#048243'
edges = np.arange(-18, -4, 0.75)
plt.hist(scp_fly_lab, bins = edges, color = lab_color, alpha = 0.5, label = 'SCP (n = ' + str(len(scp_fly_lab)) + ')')
plt.hist(cc_fly_lab, bins = edges, color = 'xkcd:reddish', alpha = 0.5, label = 'CT$_{min}$ (n = ' + str(len(cc_fly_lab)) + ')')

ax = plt.gca()
hs, ls = ax.get_legend_handles_labels()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_yticks(ticks = np.arange(0, 17, 4))

sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'cc_scp_hist_a.png'))
plt.show()


In [ ]:
# histograms of supercooling and chill coma 
sns.set()
sns.set_style('ticks')

cr_data = data_scp[data_scp.cold_room == 'y']
lab_data = data_scp[data_scp.cold_room == 'n']
scp_cp_cold_room = cr_data[~cr_data['scp_ambient_temp'].isnull()]['scp_ambient_temp'].astype(float)
scp_cp_lab = lab_data[~lab_data['scp_ambient_temp'].isnull()]['scp_ambient_temp'].astype(float)
cc_cp_lab = lab_data[~lab_data['cc_ambient_temp'].isnull()]['cc_ambient_temp']

figure = plt.figure(figsize = (6, 4), dpi = 600)
plt.title('distribution of cold tolerance thresholds', fontsize = 14)
plt.ylabel('snow fly count', fontsize = 14)
plt.xlabel('cold plate temperature ($^{\circ}$C)', fontsize = 14)
plt.xlim([-12, -4])
plt.ylim([0, 16])
cr_color = '#665fd1'
lab_color = '#048243'
edges = np.arange(-18, -4, 0.75)
plt.hist(scp_cp_lab, bins = edges, color = lab_color, alpha = 0.5, label = 'SCP (n = ' + str(len(scp_cp_lab)) + ')')
plt.hist(cc_cp_lab, bins = edges, color = 'xkcd:reddish', alpha = 0.5, label = 'CT$_{min}$ (n = ' + str(len(cc_cp_lab)) + ')')

ax = plt.gca()
hs, ls = ax.get_legend_handles_labels()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_yticks(ticks = np.arange(0, 17, 4))

sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'cc_scp_hist_b.png'))
plt.show()

fig = plt.figure(figsize = (3,3), dpi = 600)
plt.legend(handles = hs, labels = ls, fontsize = 12)
plt.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'cc_scp_legend.png'))
plt.show()

In [ ]:
# find index of the largest change in temperature to know how 
# to align the supercooling points
flyids_scp = data_scp[~data_scp['scp_frame'].isnull()].flyid
scp_ix_dict = {}
before = 1000
after = 1000
data['scp_avg_temp_increase'] = np.nan
data['scp_max_temp_increase'] = np.nan

for fly in flyids_scp: 
    
    fly_mask = data.flyid == fly
    fly_data = data.loc[fly_mask]
    col = data_scp[data_scp.flyid == fly]
    ix = int(col.iloc[0]['scp_frame'])
    start = ix - before
    end = ix + after
    mask = np.zeros(len(fly_data), dtype = bool)
    mask[start:end] = True
    avg_temp = fly_data.avg_temp_filtered[start:end]
    max_temp = fly_data.max_temp_filtered[start:end]
    cold_plate_temp = fly_data.cold_plate_temp_filtered[start:end]
    
    delta_avg_temp = np.diff(avg_temp[:1500])
    delta_max_temp = np.diff(max_temp[:1500])
    scp_ix = start + np.argmax(delta_avg_temp)
    scp_ix_dict[fly] = scp_ix
    avg_temp_increase = np.max(fly_data.avg_temp_filtered[scp_ix-100:scp_ix+100]) - np.min(fly_data.avg_temp_filtered[scp_ix-100:scp_ix+100])
    max_temp_increase = np.max(fly_data.max_temp_filtered[scp_ix-100:scp_ix+100]) - np.min(fly_data.max_temp_filtered[scp_ix-100:scp_ix+100]) 
    
    data.loc[fly_mask, 'scp_pre_avg_temp'] = np.min(fly_data.avg_temp_filtered[scp_ix-100:scp_ix+100])
    data.loc[fly_mask, 'scp_post_avg_temp'] = np.max(fly_data.avg_temp_filtered[scp_ix-100:scp_ix+100])
    data.loc[fly_mask, 'scp_pre_max_temp'] = np.min(fly_data.max_temp_filtered[scp_ix-100:scp_ix+100])
    data.loc[fly_mask, 'scp_post_max_temp'] = np.max(fly_data.max_temp_filtered[scp_ix-100:scp_ix+100])
    data.loc[fly_mask, 'scp_avg_temp_increase'] = avg_temp_increase
    data.loc[fly_mask, 'scp_max_temp_increase'] = max_temp_increase

In [ ]:
# histogram of temperature right before scp 
sns.set()
sns.set_style('ticks')

flyids_scp = data_scp[~data_scp['scp_frame'].isnull()].flyid
subset = data[data.flyid.isin(flyids_scp)]
pre_avg_temp = subset.groupby(by=['flyid']).mean().scp_pre_avg_temp
pre_max_temp = subset.groupby(by=['flyid']).mean().scp_pre_max_temp

figure = plt.figure(figsize = (9, 5))
plt.title('average temperature before the supercooling point', fontsize = 14)
plt.ylabel('number of freezing events', fontsize = 14)
plt.xlabel('temperature ($^{\circ}$C)', fontsize = 14)
edges = np.arange(-15, -1, 0.5)
plt.hist(pre_avg_temp, bins = edges, color = 'b', alpha = 0.6, label = 'average temp')
plt.hist(pre_max_temp, bins = edges, color = 'r', alpha = 0.6, label = 'max temp')
plt.legend()
sns.despine()
plt.show()

In [ ]:
# histogram of temperature right after scp 
sns.set()
sns.set_style('ticks')

flyids_scp = data_scp[~data_scp['scp_frame'].isnull()].flyid
subset = data[data.flyid.isin(flyids_scp)]
post_avg_temp = subset.groupby(by=['flyid']).mean().scp_post_avg_temp
post_max_temp = subset.groupby(by=['flyid']).mean().scp_post_max_temp

figure = plt.figure(figsize = (9, 5))
plt.title('average temperature after the supercooling point', fontsize = 14)
plt.ylabel('number of freezing events', fontsize = 14)
plt.xlabel('temperature ($^{\circ}$C)', fontsize = 14)
edges = np.arange(-12, 6, 0.5)
plt.hist(post_avg_temp, bins = edges, color = 'b', alpha = 0.6, label = 'average temp')
plt.hist(post_max_temp, bins = edges, color = 'r', alpha = 0.6, label = 'max temp')
plt.legend()
sns.despine()
plt.show()

In [ ]:
# histogram of temperature increase at scp 
sns.set()
sns.set_style('ticks')

flyids_scp = data_scp[~data_scp['scp_frame'].isnull()].flyid
subset = data[data.flyid.isin(flyids_scp)]
avg_temp_increase = subset.groupby(by=['flyid']).mean().scp_avg_temp_increase
max_temp_increase = subset.groupby(by=['flyid']).mean().scp_max_temp_increase

figure = plt.figure(figsize = (9, 5))
plt.title('temperature increase at the supercooling point', fontsize = 14)
plt.ylabel('number of freezing events', fontsize = 14)
plt.xlabel('change in temperature ($^{\circ}$C)', fontsize = 14)
edges = np.arange(0, 9, 0.5)
plt.hist(avg_temp_increase, bins = edges, color = 'b', alpha = 0.6, label = 'average temp')
plt.hist(max_temp_increase, bins = edges, color = 'r', alpha = 0.6, label = 'max temp')
plt.legend()
sns.despine()
plt.show()

In [ ]:
# for each fly that undergoes spontaneous ice nucleation, plot the 
# temperatures for a short interval surrounding the freezing event
flyids_scp = data_scp[~data_scp['scp_frame'].isnull()].flyid
before = 900
after = 900

for fly in flyids_scp: 
    
    fly_data = data[data.flyid == fly]
    t_s, t_m = get_video_length(len(fly_data), thresh = None)
    col = data_scp[data_scp.flyid == fly]
    ix = int(col.iloc[0]['scp_frame'])
    start = ix - before
    end = ix + after
    mask = np.zeros(len(fly_data), dtype = bool)
    mask[start:end] = True
    t = t_m[mask]
    avg_temp = fly_data.avg_temp_filtered[start:end]
    max_temp = fly_data.max_temp_filtered[start:end]
    cold_plate_temp = fly_data.cold_plate_temp_filtered[start:end]
    
    fig = plt.figure(figsize = (8, 4))
    plt.title(fly, fontsize = 14)
    plt.xlabel('time (minutes)', fontsize = 14)
    plt.ylabel('temperature ($^{\circ}$C)', fontsize = 14)
    
    plt.plot(t, max_temp, 'xkcd:light blue', linewidth = 1, label = 'max snowfly temp')
    plt.plot(t, avg_temp, 'xkcd:slate blue', linewidth = 1, label = 'avg snowfly temp')
    plt.plot(t, cold_plate_temp, 'xkcd:black', linewidth = 1, label = 'cold plate temp')
    
    sns.despine()
    plt.show()

In [ ]:
# for each fly that undergoes spontaneous ice nucleation, 
# plot temperature increase on the same plot
# ALIGNED IN TIME
flyids_data = data_scp[~data_scp['scp_frame'].isnull()]
cold_room_flyids = flyids_data[flyids_data.cold_room == 'y'].flyid
lab_flyids = flyids_data[flyids_data.cold_room == 'n'].flyid
dpi = 600

# flyids_scp = data_scp[~data_scp['scp_frame'].isnull()].flyid
colors = sns.cubehelix_palette(8, start=.5, rot=-.5)
before = 900
after = 900

cr_color = '#665fd1'
lab_color = '#048243'

fig = plt.figure(figsize = (6, 4), dpi = dpi)
plt.title('spontaneous ice nucleation', fontsize = 14)
plt.xlabel('time (seconds)', fontsize = 14)
plt.ylabel('average snow fly \n temperature ($^{\circ}$C)', fontsize = 14)
# plt.ylabel('detected snow fly \n temperature ($^{\circ}$C)', fontsize = 14)

for j, fly in enumerate(lab_flyids): 
    
    fly_data = data[data.flyid == fly]
    col = data_scp[data_scp.flyid == fly]
    ix = scp_ix_dict[fly]
    start = ix - before
    end = ix + after
    t = np.arange(0, abs(end - start), 1)
    t = t / 30
    avg_temp = fly_data.avg_temp_filtered[start:end]
    max_temp = fly_data.max_temp_filtered[start:end]
    
    plt.plot(t, avg_temp, color = lab_color, linewidth = 0.5, label = 'lab', alpha = 0.8)
    # plt.plot(t, max_temp, color = lab_color, linewidth = 0.5, label = 'lab', alpha = 0.8)
    
for j, fly in enumerate(cold_room_flyids): 
    
    fly_data = data[data.flyid == fly]
    col = data_scp[data_scp.flyid == fly]
    ix = scp_ix_dict[fly]
    start = ix - before
    end = ix + after
    t = np.arange(0, abs(end - start), 1)
    t = t / 30
    avg_temp = fly_data.avg_temp_filtered[start:end]
    max_temp = fly_data.max_temp_filtered[start:end]
    
    plt.plot(t, avg_temp, color = cr_color, linewidth = 0.5, label = 'cold room', alpha = 0.8)
    # plt.plot(t, max_temp, color = cr_color, linewidth = 0.5, label = 'cold room', alpha = 0.8)

ax = plt.gca()
hs, ls = ax.get_legend_handles_labels()

sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'scp_avg_temp_aligned.png'))
# plt.savefig(os.path.join(savepath, 'scp_max_temp_aligned.png'))
plt.show()

fig = plt.figure(figsize = (3,3), dpi = dpi)
legend_dict = dict(zip(ls, hs))
plt.legend(legend_dict.values(), legend_dict.keys())
plt.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'scp_temp_legend.png'))
plt.show()

In [ ]:
# for each fly that undergoes spontaneous ice nucleation, 
# plot temperature increase on the same plot
# ALIGNED IN TIME AND BASELINE SUBTRACTED
flyids_data = data_scp[~data_scp['scp_frame'].isnull()]
cold_room_flyids = flyids_data[flyids_data.cold_room == 'y'].flyid
dpi = 600 

# flyids_scp = data_scp[~data_scp['scp_frame'].isnull()].flyid
colors = sns.cubehelix_palette(8, start=.5, rot=-.5)
before = 900
after = 900

cr_color = '#665fd1'
lab_color = '#048243'

fig = plt.figure(figsize = (6, 4), dpi = dpi)
plt.title('spontaneous ice nucleation', fontsize = 14)
plt.xlabel('time (seconds)', fontsize = 14)
plt.ylabel('change in average \n snow fly temperature ($^{\circ}$C)', fontsize = 14)
# plt.ylabel('change in detected \n snow fly temperature ($^{\circ}$C)', fontsize = 14)

for j, fly in enumerate(lab_flyids): 
    
    fly_data = data[data.flyid == fly]
    col = data_scp[data_scp.flyid == fly]
    ix = scp_ix_dict[fly]
    start = ix - before
    end = ix + after
    t = np.arange(0, abs(end - start), 1)
    t = t / 30
    avg_temp = fly_data.avg_temp_filtered[start:end]
    avg_temp_aligned = avg_temp - avg_temp.iloc[0]
    max_temp = fly_data.max_temp_filtered[start:end]
    max_temp_aligned = max_temp - max_temp.iloc[0]
    
    plt.plot(t, avg_temp_aligned, color = lab_color, linewidth = 0.5, alpha = 0.8)
    # plt.plot(t, max_temp_aligned, color = lab_color, linewidth = 0.5, alpha = 0.8)
    
for j, fly in enumerate(cold_room_flyids): 
    
    fly_data = data[data.flyid == fly]
    col = data_scp[data_scp.flyid == fly]
    ix = scp_ix_dict[fly]
    start = ix - before
    end = ix + after
    t = np.arange(0, abs(end - start), 1)
    t = t / 30
    avg_temp = fly_data.avg_temp_filtered[start:end]
    avg_temp_aligned = avg_temp - avg_temp.iloc[0]
    max_temp = fly_data.max_temp_filtered[start:end]
    max_temp_aligned = max_temp - max_temp.iloc[0]
    
    plt.plot(t, avg_temp_aligned, color = cr_color, linewidth = 0.5, alpha = 0.8)
    # plt.plot(t, max_temp_aligned, color = cr_color, linewidth = 0.5, alpha = 0.8)

sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'scp_avg_temp_aligned_baseline.png'))
# plt.savefig(os.path.join(savepath, 'scp_max_temp_aligned_baseline.png'))
plt.show()

In [ ]:
# # histograms of chill coma and chill coma recovery temperatures  
# sns.set()
# sns.set_style('ticks')

# figure = plt.figure(figsize = (9, 5))
# plt.title('Distribution of snow fly temperatures', fontsize = 14)
# plt.ylabel('Counts', fontsize = 14)
# plt.xlabel('Temperature ($^{\circ}$C)', fontsize = 14)
# plt.xlim([-15, 15])

# edges = np.arange(-15, 15, 1.5)
# #plt.hist(cc_ambient, bins = edges, color = colors[0], alpha = 0.8, label = 'cold plate temp')
# plt.hist(cc_fly, bins = edges, color = 'k', alpha = 0.8, label = 'Critical thermal minimum ($CT_{min}$)')
# #plt.hist(recovery_ambient, bins = edges, color = colors[2], alpha = 0.8, label = 'cold plate temp (recovery)')
# plt.hist(recovery_fly, bins = edges, color = '#999999', alpha = 0.8, label = 'Chill coma recovery temperature')

# plt.legend(fontsize = 13, loc = 'upper right')
# sns.despine()
# plt.show()

In [ ]:
# histograms of chill coma and supercooling 
sns.set()
sns.set_style('ticks')

figure = plt.figure(figsize = (8, 5))
plt.title('Distribution of snow fly temperatures', fontsize = 14)
plt.ylabel('Counts', fontsize = 14)
plt.xlabel('Temperature ($^{\circ}$C)', fontsize = 14)
plt.xlim([-14, 2])
plt.ylim([0, 20])

edges = np.arange(-14, 2, 1)
plt.hist(scp_fly_pre, bins = edges, color = 'r', alpha = 0.5, label = 'Supercooling point \n (n = ' + str(len(scp_fly_pre)) + ')')
plt.hist(cc_fly, bins = edges, color = 'b', alpha = 0.5, label = 'Critical thermal minimum \n (n = ' + str(len(cc_fly)) + ')')

ax = plt.gca()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_yticks(ticks = np.arange(0, 21, 4))

plt.legend(fontsize = 13, loc = 'upper right')
sns.despine()
plt.show()

In [ ]:
# # histograms of supercooling points 
# sns.set()
# sns.set_style('ticks')
# colors = sns.color_palette('mako', 4)

# figure = plt.figure(figsize = (9, 5))
# plt.title('Distribution of supercooling points in snowflies', fontsize = 14)
# plt.ylabel('Counts', fontsize = 14)
# plt.xlabel('Temperature ($^{\circ}$C)', fontsize = 14)

# edges = np.arange(-20, 2, 1)
# scp_delta = abs(abs(scp_fly_pre) - abs(scp_fly_post))
# plt.hist(scp_ambient, bins = edges, color = 'k', alpha = 0.85, label = 'Cold plate temperature')
# plt.hist(scp_fly_pre, bins = edges, color = '#999999', alpha = 0.85, label = 'Snow fly temperature (pre-SCP)')
# plt.hist(scp_fly_post, bins = edges, color = 'w', edgecolor = 'k', alpha = 0.3, label = 'Snow fly temperature (post-SCP)')
# # plt.hist(scp_delta, bins = edges, color = colors[3], alpha = 0.85, label = 'Snow fly temp increase')

# plt.legend(fontsize = 13, loc = 'upper left')
# sns.despine()
# plt.show()

In [ ]:
# scatterplot of supercooling point vs elevation
sns.set_style('ticks')

subset = data_scp[~data_scp['scp_frame'].isnull() & ~data_scp['elevation'].isnull()]
scp_temps = subset['scp_fly_temp_pre']
elevations = subset['elevation']

fig = plt.figure()
plt.xlabel('elevation (ft)')
plt.ylabel('supercooling point ($^{\circ}$C)')
plt.scatter(elevations, scp_temps)

sns.despine()
plt.show()